# <center> Basics of Mobile Robotics Project -  Report Notebook <center>
### Catarina Pires 
    
Some Spanish girl
    
### Dimitri Hollosi 247227 
    
MA3 student in Mechanical Engineering, specialising in automation & control systems
    
### Goncalo Gomes 
    
Some Spanish guy
    
### Richard Gao 342177

MA1 student in Robotics, specialising in mobile robotics and data science


## General Considerations and  Introduction to the Environment


Here the main code structure and functionalities aim to be detailed so as to gain an understanding in the underlying structure behind the Thymio implementations. Generally speaking, the code can be split into 3 main parts, namely Camera Vision, Global Navigation and Local Navigation. The latter includes all aspects related to obstacle avoidance, which was iteratively tuned to obtain desired performance. One important element to consider prior to delving into the implemented functionalities within the code would be the chosen environment setup for the final project implementation. Indeed, it was decided to use one of the team member's lower wardrobe portion so as to benefit from a relatively "stable" environment, which could further simplifiy the calibration, lighting and initial setup process. This environment does however come with its limitations, as this implies that the Thymio must navigate in a restricted and confined space, namely implying that the local navigation must take that into account (i.e walls and corners must be accounted for) and yield a relatively high responsiveness.

### Code Structure

Wait for final main to write about this

## Import Required Libraries and Functions

To run this code we must first import several libraries. The first three lines of the code cell below are easily recognisable. The Local_Nav and Global_Nav files were written to contain the class objects which control local and global navigation respectively. The sys library is imported as it allows us to change the recursion limit on the operating system which is useful for better image detection and path finding.

In [ ]:
import numpy as np
import cv2 as cv
from tdmclient import ClientAsync
from Local_Nav import Local_Nav as LN
from Global_Nav import Global_Nav 
import sys
sys.setrecursionlimit(5000)

## Set Thymio Parameters and Global Variables

In the code cell below global constants concerning the Thymio are initialised. These constants are used to help calculate Thymio speed, angular velocity of the wheels and set the design or "cruising" speed. Another constant, pixel_to_cm, is also intitialised to convert distances seen on the camera into cm distances. These variables are sent to the Global_Nav class to be used as class attributes, they are intialised here so as to be more easily accesible for modifications to be made.

In [ ]:
r = 2                                               # radius of the Thymio wheels
L = 9.5                                             # distance between the Thymio wheels
v_max = 20                                          # max linear velocity of the thymio in cm/s
cruising = 150                                      # the design speed of the Thymio in this project in Thymio speed units
motor_speed_max = 500                               # the max motor speed of the Thymio in Thymio speed units
omega_to_motor = (motor_speed_max*r)/v_max          # the conversion factor from ?
pixel_to_cm = 1/6                                   # the conversion factor for the pixels in the video to cm in the environment

## PID Controller

In order to reach it's goal, the Thymio must convert the difference between it's current position and orientation and that of it's goal into a control action. A PID controller is used here which produces a control action that is proportional to this difference, damped so that the control action does not oscillate due to overshooting and corrected for accumulated errors. This method was chosen over a simpler proportional control as it was found through testing that the damping and error correction provided by the derivative and integral terms are necessary for the Thymio to navigate with higher precision in the limited space available to us. The gains for the controller are initialised in the code cell below along with other variables that are used in the calculation of the control action. The values of **Kp**, **Kd** and **Ki** were found through manual tuning of the controller until a desired output was shown. The **previous_error** is used to calculate the error difference between time steps and hence the damping term.

The PIDcontroller function can be found as a class method in the Global_Nav.py file. This function takes as input the angle between the vector created by the midpoint of the Thymio to the goal and the vector that runs straight down the centre of the Thymio. The output of the function is the angular velocity of the left and right wheels of the Thymio which are sent to the Thymio at each timestep. The goal of the function is to minimise the angle, thought of as the error, thus turning the Thymio towards the goal. The error is converted to an angular velocity by applying the gains as described in the equation below. To create the control action, the angular velocity of each wheel is described in terms of a design speed and the angular velocity created by the error. The error term is added or subtracted to the design speed of each wheel so as to have an opposite effect on each wheel thereby causing the Thymio to rotate in order to minimise the error and move the Thymio toward the goal at the design speed. 

<center> $\omega$ = Kp * error + Ki * $\Sigma$ error - Kd * (error - previous_error) </center>


In [3]:
Kp = 0.7                      # Proportional gain 
Ki = 0.02                     # Integral gain
Kd = 0.1                      # Derivative gain
previous_error = 0            # initialised at 0 and updated for each time step

## Set Kalman Filter Parameters

In complex systems such as mobile robots, the state of the sysem is almost impossible to determine with complete accuracy. Sensor data of the system can be used to provide a better estimation of the system state and typically the data is passed through a filter when the variables of interest cannot be measured directly. In the case of the Thymio, it's position and orientation can be reliably found using the camera but, when the camera is obscured, we must rely on the Thymio motor speed sensors to infer the new position and orientation at each timestep. Several types of filter exist but the Kalman filter is widely used in robotics for tracking and prediction applications as it is well suited to systems with uncertainty about an observable dynamic process i.e. the motor speeds. 

The Kalman filter is programmed as a recursive function using the estimated state from the previous time step and the sensor data to estimate the current state. At each time step the state transition matrix **A** is used to predict the state variables at the current time step from their previous values. The state covariance **P**, initialised as **P0**, is also predicted using **A**. The observed sensor data at the current time step is then used to calculate the Kalman gain in conjunction with the error in the measurement **R** and the error in the estimate **Q**. Finally the Kalman gain is used to update the prediction of the state and the covariance towards a belief that lies between the initial prediction and the measurement.

The variables used in the Kalman filter function are initialised in the code cell below and the function itself can be found as a class method in the Global_Nav.py file. The covariance function used to create the covariance matrices can also be found in this file. The state transition matrix **A** is constructed so as to progress the coordinates and orientation of the Thymio by a single timestep given their previous rates of change. The covariance matrices are constructed so that each diagonal element is an error corresponding to the state variable which it relates to. These initial values are chosen from the sample values given in the Week 8 Exercise session on Kalman filter implementation and have proved to be sufficient.

In [ ]:
ts = 0.1                                                           # time step (s)

A = np.array([[1, 0, 0, 0, 0, 0],     # xdot                       # state transition matrix
              [ts, 1, 0, 0, 0, 0],    # x
              [0, 0, 1, 0, 0, 0],     # ydot
              [0, 0, ts, 1, 0, 0],    # y
              [0, 0, 0, 0, 1, 0],     # phidot
              [0, 0, 0, 0, ts, 1]])   # phi

P0 = GN.covariance(100, 100, 100, 100, 100, 100)                   # state covariance matrix

Q = GN.covariance(0.0615, 0.004, 0.0615, 0.004, 0.0615, 0.004)     # process noise covariance matrix

R = GN.covariance(0.0615, 0.025, 0.0615, 0.025, 0.0615, 0.025)     # measurement covariance matrix

## Begin Thymio Connection and Initialise the Global Navigation Class Object

Client and node explanation

Global Nav explanation

In [ ]:
client = ClientAsync()
node = client.aw(client.wait_for_node())

# Define Global_Nav object with class attributes
GN = Global_Nav(client, node, cruising, ts, Kp, Kd, previous_error, r, L, omega_to_motor, pixel_to_cm, R, Q, A, P0)

## Vision and Path Creation

Use this cell to describe how we filter/augment the camera image and how the shortest path is created. Make sure to explain why these methods are chosen.

In [ ]:
"""Use this cell to write the code for Vision and Path Creation"""

## Controlling the Thymio and Going to the Goal

Use this cell to describe how the Thymio is controlled in the "main" loop. We might split this into more sections.

In [ ]:
"""Use this cell to write the code for actually sending the Thymio to the goal"""